# Algorytm FFT

In [39]:
using WAV
using Random
using FFTW

In [40]:
function fft_b(x)
    N = length(x)
    if log2(N) % 1 > 0
        throw(ArgumentError("must be a power of 2"))
    end
    N_min = min(N, 1)
    
    n = collect(0:(N_min-1))
    M = exp.(-2π * im * n * n'/ N_min)
    X = M * reshape(x, (:, N_min))'
    while size(X)[1] < N
        X_even = X[:, 1:Int(size(X)[2] / 2)]
        X_odd = X[:, (Int(size(X)[2] / 2)+1):end]
        t = exp.(-1*im * π * collect(0:(size(X)[1]-1))/size(X)[1])
        terms = hcat(map(x->t, 1:size(X_odd)[2])...)
        X = vcat(X_even + terms .* X_odd, X_even - terms .* X_odd)
    end
    return vec(X)
end
        
        
    

fft_b (generic function with 1 method)

## Przykład 1

In [41]:
@time p1_n = fft_b([1,2,3,4, 5, 6, 7, 8]);

  0.044563 seconds (735 allocations: 40.484 KiB)


In [42]:
@time p1_o =fft([1,2,3,4, 5, 6, 7, 8]);

  0.000107 seconds (36 allocations: 2.984 KiB)


In [43]:
isapprox(p1_n, p1_o)

true

## Przykład 2

In [44]:
p2 = rand(1024);

In [45]:
@time p2_n = fft_b(p2);

  0.183617 seconds (153.60 k allocations: 8.690 MiB)


In [46]:
@time p2_o = fft(p2);

  0.000207 seconds (35 allocations: 34.688 KiB)


In [47]:
isapprox(p2_n, p2_o)

true

## Przykład 3

In [48]:
p3 = rand(1024^2);

In [49]:
@time p3_n = fft_b(p3);

  1.673212 seconds (1.28 k allocations: 1.508 GiB, 31.86% gc time)


In [50]:
@time p3_o = fft(p3);

  0.121848 seconds (37 allocations: 32.003 MiB, 16.15% gc time)


In [51]:
isapprox(p3_n, p3_o)

true

## Przykład 4

In [52]:
bird = wavread("bird.wav")

([0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0], 48000.0f0, 0x0010, WAVChunk[WAVChunk(Symbol("fmt "), UInt8[0x10, 0x00, 0x00, 0x00, 0x01, 0x00, 0x02, 0x00, 0x80, 0xbb, 0x00, 0x00, 0x00, 0xee, 0x02, 0x00, 0x04, 0x00, 0x10, 0x00]), WAVChunk(:LIST, UInt8[0x49, 0x4e, 0x46, 0x4f, 0x49, 0x41, 0x52, 0x54, 0x0f, 0x00  …  0x35, 0x38, 0x2e, 0x34, 0x35, 0x2e, 0x31, 0x30, 0x30, 0x00])])

In [53]:
wavplay("bird.wav")

In [54]:
sound = bird[1][:,1]
wind = 1024
len = length(sound)
st = 300
ile_sampli  = (len-wind) ÷ st
slices = Array{Any}(undef,ile_sampli,wind)
for i in 1:ile_sampli
    slices[i, 1:wind] = sound[1+(i-1)*st : wind + (i-1)*st]
end

In [57]:
fft(slices)

LoadError: [91mMethodError: no method matching plan_fft(::Array{Any,2}, ::UnitRange{Int64})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  plan_fft([91m::StridedArray{T, N}[39m, ::Any; flags, timelimit) where {T<:Union{Complex{Float32}, Complex{Float64}}, N} at C:\Users\THINK\.julia\packages\FFTW\G3lSO\src\fft.jl:681[39m
[91m[0m  plan_fft([91m::AbstractArray{var"#s17",N} where N where var"#s17"<:Real[39m, ::Any; kws...) at C:\Users\THINK\.julia\packages\AbstractFFTs\JebmH\src\definitions.jl:199[39m
[91m[0m  plan_fft([91m::AbstractArray{var"#s26",N} where N where var"#s26"<:(Complex{var"#s27"} where var"#s27"<:Union{Integer, Rational})[39m, ::Any; kws...) at C:\Users\THINK\.julia\packages\AbstractFFTs\JebmH\src\definitions.jl:201[39m
[91m[0m  ...[39m

## Próba IFFT

In [25]:
function ifft_b(x)
    N = length(x)
    if log2(N) % 1 > 0
        throw(ArgumentError("must be a power of 2"))
    end
    N_min = min(N, 1)
    
    n = collect(0:(N_min-1))
    M = exp.(2π * im * n * n'/ N_min)
    X = (1/N) * M * reshape(x, (:, N_min))'
    while size(X)[1] < N
        X_even = X[:, 1:Int(size(X)[2] / 2)]
        X_odd = X[:, (Int(size(X)[2] / 2)+1):end]
        t = exp.(2*im * π * collect(0:(size(X)[1]-1))/size(X)[1])
        terms = hcat(map(x->t, 1:size(X_odd)[2])...)
        X = vcat(X_even + terms .* X_odd, X_even - terms .* X_odd)
    end
    return vec(X)
end

ifft_b (generic function with 1 method)

In [ ]:
a = fft(rand(8))

In [ ]:
b = ifft(a)